In [1]:
# Import necessary libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Check TensorFlow version
print(tf.__version__)

2.14.0


In [4]:
# Set datasets

train_data = 'dataset/Training'
test_data = 'dataset/Testing'

### Define image data generators

**rescale=1./255:**

- **Purpose**: This rescales the pixel values of the images from a range of [0, 255] to [0, 1].
- **Reason**: Neural networks perform better when input data is normalized. Scaling the pixel values to this range helps in faster convergence during training.

**rotation_range=40:**

- **Purpose**: Randomly rotates images within a range of 0 to 40 degrees.
- **Reason**: Rotation helps the model become invariant to the orientation of the images, making it more robust to variations in the dataset.

**width_shift_range=0.2:**

- **Purpose**: Shifts the image horizontally by up to 20% of the width.
- **Reason**: Horizontal shifts help the model to learn that objects can appear in different horizontal positions, enhancing its generalization capability.

**height_shift_range=0.2:**

- **Purpose**: Shifts the image vertically by up to 20% of the height.
- **Reason**: Vertical shifts serve the same purpose as horizontal shifts but in the vertical direction, improving the model's robustness to vertical translations.

**shear_range=0.2:**

- **Purpose**: Applies a shear transformation to the images, up to 20%.
- **Reason**: Shearing distorts the image along an axis, helping the model handle slight distortions or slanting of the objects within the images.

**zoom_range=0.2:**

- **Purpose**: Randomly zooms into the images by up to 20%.
- **Reason**: Zooming helps the model become invariant to the scale of the objects in the images, allowing it to detect objects at different zoom levels.

**horizontal_flip=True:**

- **Purpose**: Randomly flips the images horizontally.
- **Reason**: Horizontal flipping augments the dataset by generating mirrored versions of the images, which helps the model learn that objects can appear mirrored.

**fill_mode='nearest':**

- **Purpose**: Specifies the strategy for filling in new pixels that may be created during transformations (e.g., after rotation or shifts).
- **Reason**: 'nearest' mode fills in the new pixels with the nearest pixel value, which helps maintain the integrity of the image after augmentation.


In [6]:
# Define image data generators

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)